In [2]:
import numpy as np
import pandas as pd
import biom

In [3]:
datafile='16S.shared.biom'
sample_metadata='16S.shared.metadata.txt'
target_field='BL_AGE'
prev_threshold=0.1

In [4]:
bt=biom.load_table(datafile)
md=pd.read_csv(sample_metadata, sep='\t', index_col=0)

FileNotFoundError: [Errno 2] No such file or directory: '16S.shared.biom'

### filter sample ids that have no matched ones in the metadata

In [5]:
new_bt = bt.filter(md.index.astype(str), axis='sample', inplace=False)
new_bt

NameError: name 'bt' is not defined

### filter sample ids that have invalid values (such as `NaN`, `Not applicable`) for prediction

In [1]:
null_sum=md[target_field].isnull().sum()
print('The number of NaN in the '+target_field+": "+ str(null_sum))

NameError: name 'md' is not defined

In [6]:
y=md[target_field].to_list()
md['SampleID']=md.index

In [7]:
df=new_bt.to_dataframe(dense=True)

In [8]:
dfT=df.T

In [9]:
dfT.index.set_names('SampleID')

Index(['820003753-8', '820003769-1', '820003785-0', '820003801-3',
       '820003817-9', '820003849-9', '820003865-3', '820003881-8',
       '820003897-2', '820003913-0',
       ...
       '820252655-5', '820252665-9', '820252675-8', '820252695-6',
       '820252705-4', '820252715-0', '820252725-1', '820252745-3',
       '820252755-6', '820252765-5'],
      dtype='object', name='SampleID', length=6430)

### filter features by prevalence

In [10]:
total_rows = len(dfT.axes[0])
total_cols =len(dfT.axes[1])
prev = dfT.astype(bool).sum(axis=0)/total_rows

In [11]:
prev_criteria=(prev > prev_threshold).to_list()
len(prev_criteria)

6018

In [12]:
dfT_prev=dfT.iloc[:,prev_criteria]

In [13]:
dfT_prev[target_field]=y

/Users/huangshi/anaconda3/envs/qiime2-2019.7/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://urldefense.com/v3/__http://pandas.pydata.org/pandas-docs/stable/indexing.html*indexing-view-versus-copy*5Cn__;IyU!!Mih3wA!Sp0h-DXuSCfxP8hJbZB_d2rcelGaC59uJKurN56AMBjLoKvaesIi00yBy-wJt1wU$   """Entry point for launching an IPython kernel.


In [14]:
dfT_prev.shape

(6430, 410)

In [15]:
dfT_prev.to_csv(datafile+'.t.'+target_field+'.prev_threshold_'+str(prev_threshold)+'.csv', sep=',', index=True)